In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
import uvicorn
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from fastapi import FastAPI
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain import PromptTemplate

In [3]:
chat_history = []

def chatbot(query: str):
    global chat_history
    
    search = GoogleSearchAPIWrapper(k=3)
    tools = Tool(
        name="Google Search",
        description="Search Google for recent results related to Python and React and provide a descriptive answer based on observation. Also provide guidance to the user. Based on your observation provide reply to the user.",
        func=search.run,
    )
    llm = ChatOpenAI(temperature=0.15, model_name="gpt-3.5-turbo-0613")

    # Create the conversation buffer memory
    memory = ConversationBufferMemory(memory_key="chat_history")

    # Define the chatbot prompt template
    QA_PROMPT_TMPL = '''{bot_name} is a chatbot developed by {company_name} that helps you with Python and React-related questions. It provides users with documentation, installation assistance, code understanding, and other guidance related to Python and React. Queries non-related to python or react will never be answered by this {bot_name} and will be provided a warm reply to the user regarding this ethical stand-point. {bot_name} should not include own thought "Thought:Do I need to use a tool?" in the reply. If you do not know the answer simply provide user with a warm and friendly reply.
    Context:
    {context_str}
    Conversation History:
    {chat_history}
    User:
    {query_str}
    Reply:'''

    prompt = PromptTemplate(
        template=QA_PROMPT_TMPL,
        input_variables=["context_str", "chat_history", "query_str"],
        partial_variables={"bot_name": "codingBOT", "company_name": "XYZ"}
    )

    # Initialize the chatbot agent
    agent_chain = initialize_agent(
        [tools],
        llm,
        agent="conversational-react-description",
        memory=memory,
        verbose=True,
        max_iterations=5,
    )

    # Generate the chatbot response
    chat_history_str = "\n".join(chat_history)
    agent_prompt = prompt.format(
        query_str=query,
        context_str="casual discussion between user and bot about Python or React",
        chat_history=chat_history_str
    )
    
    try:
        response = agent_chain.run(input=agent_prompt)
    except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output: `"):
            response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

    # Add the user's query and the chatbot's response to the chat history
    chat_history.append(f"User: {query}, Response: {response}")

    if len(chat_history_str) > 3000:
        chat_history.pop(0)
    return response

query = "tell me the latest version of pandas"
response = chatbot(query)
print(response)



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Google Search
Action Input: latest version of pandas
Observation: pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, ... Install pandas now! ... Latest version: 2.0.2. Binary installers for the latest released version are available at the Python Package Index (PyPI) and on Conda. # conda conda install pandas # or PyPI pip ... Release notes#. This is the list of changes to pandas between each release. For full details, see the commit logs. For install and upgrade instructions, ...
Thought:Do I need to use a tool? Yes
Action: Google Search
Action Input: latest version of pandas
Observation: pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, ... Install pandas now! ... Latest version: 2.0.2. Binary installers for the latest released version are available at the Python Package Index (PyPI) and on Conda. # conda conda 